In [ ]:
# import gmt gravfft (calculate gravity effect of bathymetry-seamount interface)
# import simpeg 
# import choclo (Fatiando a Terra, 2024) (calculate components of gravitational tensor for the synthetic model)
# import grdflexure GMT module (wessel, 2019) (calculate flexural deformation of lithosphere)

## Outline
- download bathymetry data from GEBCO
- make model of seamount in the shape of Kolumbo volcano
- introduce magma chamber in location, size and density corresponding to results from previous seismic tomography studies
- calculate gravity gradient effect of bathymetry (use different data set for bathymetry than satellite altimetry, to ensure independence) using Parker's expansion for a range of reference crustal densities
- filter 
- subtract bathymetry gravity effect from satellite altimetry gravity gradient
- model flexural compensation of seamount ()


find out
-read seismo papers on Kolumbo volcano

